In [9]:
#Here we will find a conventional treatment plan for real patient to make sure it makes sense
#Also, will check and incorporate smoothing

In [6]:
import numpy as np
import scipy.optimize
import pandas as pd
import operator
import scipy.io
import scipy
import scipy.sparse
import time
import sys

In [8]:
sys.path.append('/home/rilevin/MMORT/mmort')
from optimization_tools import *
from experiments import *